In [1]:
import os
from keras.preprocessing import image
import cv2

In [2]:
categories = ['1','0']


In [3]:
data = []
for category in categories:
    path=os.path.join('dataset',category)
#     print(path)
    label= categories.index(category)
    
    for file in os.listdir(path):
#         print(file)
#         break
        img_path = os.path.join(path,file)
        img=cv2.imread(img_path)
        if img is None:
            continue
        img=cv2.resize(img,(224,224))
        data.append([img,label])

In [4]:
len(data)

1376

In [5]:
import random

random.shuffle(data)

In [6]:
X = []
y = []
for features,label in data:
    X.append(features)
    y.append(label)

In [7]:
import numpy as np
len(y)

1376

In [8]:
X = np.array(X)
y = np.array(y)

In [9]:
X.shape

(1376, 224, 224, 3)

In [10]:
y.shape

(1376,)

In [11]:
y

array([1, 1, 1, ..., 1, 1, 0])

In [12]:
X=X/255

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [14]:
X_train.shape

(1100, 224, 224, 3)

In [15]:
X_test.shape

(276, 224, 224, 3)

In [16]:
from keras.applications.vgg16 import VGG16

In [17]:
vgg=VGG16()

In [18]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
from keras import Sequential

In [20]:
model = Sequential()

In [31]:
for layer in vgg.layers[:-1]:
    model.add(layer)

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


ValueError: Exception encountered when calling layer "block1_conv1" (type Conv2D).

Shape must be rank 4 but is rank 2 for '{{node block1_conv1/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, block1_conv1/Conv2D/ReadVariableOp)' with input shapes: [?,1], [3,3,3,64].

Call arguments received by layer "block1_conv1" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 1), dtype=float32)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [23]:
for layer in model.layers:
    layer.trainable=False

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [25]:
from keras.layers import Dense

In [26]:
model.add(Dense(1,activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [28]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=["AUC","accuracy"])

In [30]:
model.fit(X_train,y_train,epochs=3,validation_data=(X_test,y_test))

Epoch 1/3
35/35 [==============================] - 238s 7s/step - loss: 0.5296 - auc: 0.8242 - accuracy: 0.7164 - val_loss: 0.3552 - val_auc: 0.9763 - val_accuracy: 0.9203
Epoch 2/3
35/35 [==============================] - 239s 7s/step - loss: 0.3229 - auc: 0.9695 - accuracy: 0.8955 - val_loss: 0.3020 - val_auc: 0.9830 - val_accuracy: 0.8913
Epoch 3/3
35/35 [==============================] - 238s 7s/step - loss: 0.2491 - auc: 0.9839 - accuracy: 0.9318 - val_loss: 0.2187 - val_auc: 0.9868 - val_accuracy: 0.9348


In [40]:
cap = cv2.VideoCapture(0)

In [43]:
while True:
    
    ret, frame=cap.read()
    
    # call the detection method
    frame = cv2.resize(frame,(224,224))
    
    y_pred= detect_face_mask(frame)
    
    if(y_pred==1):
        draw_label(frame,"No_mask",(30,30),(0,0,255))
    else:
        draw_label(frame,"mask",(30,30),(0,255,0))
    
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

1/1 [==============================] - 0s 91ms/step


1/1 [==============================] - 0s 95ms/step


1/1 [==============================] - 0s 99ms/step


1/1 [==============================] - 0s 222ms/step


1/1 [==============================] - 0s 226ms/step


1/1 [==============================] - 0s 219ms/step


1/1 [==============================] - 0s 214ms/step


KeyboardInterrupt: 

In [42]:
def detect_face_mask(img):
    y_pred=model.predict(img.reshape(1,224,224,3))
    y_pred = (y_pred > 0.5).astype("int32")
    return y_pred

In [34]:
def draw_label(img,text,pos,bg_color):
    text_size =cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0] +text_size[0][0] + 2
    end_y=pos[1] +text_size[0][1] - 2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

NameError: name 'history' is not defined

<Figure size 640x480 with 0 Axes>